In [40]:
from tinygrad import Tensor
from tinygrad.ops import BinaryOps, ReduceOps
from tinygrad.engine.schedule import create_schedule
from tinygrad.engine.realize import run_schedule

In [94]:
a = Tensor([1, 2, 3], requires_grad=True)
print(a)
b = a.max(axis=0)
print(b.numpy())
c = b.backward()
print(c.grad.numpy(), a.grad.numpy(), b.grad.numpy())

<Tensor <LB METAL (3,) contig:True (<LoadOps.COPY: 3>, None)> on METAL with grad None>
3
1 [0 0 1]


In [81]:
axis = (0,)
x = a.lazydata

ret = x.r(ReduceOps.MAX, axis)
sched = create_schedule([ret])
run_schedule(sched)
print(ret.realized.as_buffer().cast('I')[0])

max_is_1s = x.e(BinaryOps.CMPEQ, ret.expand(x.shape)).cast(x.dtype)
div = max_is_1s.r(ReduceOps.SUM, axis).expand(x.shape)
res = max_is_1s.e(BinaryOps.DIV, div).e(BinaryOps.MUL, ret.expand(x.shape))
sched = create_schedule([max_is_1s, div, res])
run_schedule(sched)
print(max_is_1s.realized.as_buffer().cast('I')[2])
print(div)
print(res.realized.as_buffer().cast('I')[2])

3
1
<LB METAL (3,) contig:False ShapeTracker(views=(View(shape=(3,), strides=(0,), offset=0, mask=None, contiguous=False),))>
3


In [93]:
a = Tensor([[1, 2, 3], [4, 5, 6]])
b = a.max(axis=0)
print(b.numpy(), b.expand(a.shape).numpy())
# max_is_1s = a
# div = max_is_1s.sum(axis=0).expand(a.shape)
# print(div.numpy())
# res = max_is_1s.div(div).mul(b.expand(a.shape))
# print(b.expand(a.shape).numpy())
# print(res.numpy())


AssertionError: can't expand (3,) into (2,)